<a href="https://colab.research.google.com/github/u9999yoko/colab/blob/main/anonym_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Colaboratoryを便利に使うためのTIPSまとめ
https://karaage.hatenadiary.jp/entry/2018/12/17/073000#%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E3%82%A2%E3%83%83%E3%83%97%E3%83%AD%E3%83%BC%E3%83%89%E3%83%80%E3%82%A6%E3%83%B3%E3%83%AD%E3%83%BC%E3%83%89

In [ ]:
%%time
!pip install pandas regex

CPU times: user 106 ms, sys: 15.5 ms, total: 122 ms
Wall time: 12.8 s


In [ ]:
!pip install ja_ginza --break-system-packages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 19.1 MB/s eta 0:00:00


https://qiita.com/itsuki3/items/22e7d7a91ec820ab654e

In [ ]:
import os
import re
import random
import string
import pandas as pd
from pathlib import Path

In [ ]:
# 市区町村コード一覧（20240101現在の団体.csv）がなかったらアップロードする
from google.colab import files
org_file = './20240101現在の団体.csv'
is_file = os.path.isfile(org_file)
if is_file:
  print('file exist')
else:
  uploaded = files.upload()

import glob
# 特定のパターンに一致するファイルを検索
txt_files = glob.glob("2*.txt")
print("テキストファイルの一覧:")
for file in txt_files:
    print(f"{file}")

file_txt = txt_files[0]
print(file_txt)
out_file_txt = file_txt.replace(".txt", "_anonym.txt")
print(out_file_txt)

# pathlib_path = Path("/content/")
# file_list_txt =glob.glob(r'2*.txt', recursive=True)
# is_file = file_list_txt[0]
# if is_file:
#   print('file exist')
# else:
#   uploaded = files.upload()

# # クリーンアップ
# for file in glob.glob("**/*.txt", recursive=True):
#     os.remove(file)
# os.rmdir("subdir")
# print("\nテスト用ファイルとディレクトリを削除しました")

file exist
テキストファイルの一覧:
20240722_1004JATA.txt
20240722_1004JATA.txt
20240722_1004JATA_anonym.txt


In [ ]:
import re
import secrets
import string

def generate_secure_random_string(length=12):
    """Generate a secure random string of specified length."""
    alphabet = string.ascii_letters + string.digits
    return ''.join(secrets.choice(alphabet) for i in range(length))

def anonymize_with_pattern(patterns, text):
    """
    Replaces matches of patterns with random strings,
    excluding those in brackets with specific time formats.
    """

    # 1. Protect numbers inside brackets with a temporary placeholder
    # (Before replacing brackets with special symbols)
    bracketed_numbers = re.findall(r"(?:\d+[:.]\d+|\d+[hms])", text) # Changed pattern to match original brackets
    # print("aaa", bracketed_numbers)
    for i, num in enumerate(bracketed_numbers):
        text = text.replace(num, f"[_PROTECTED_NUMBER_{i}]", 1)  # Changed placeholder format

    # 2. Replace brackets with special symbols to avoid regex conflicts
    text = text.replace("[", "｟")
    text = text.replace("]", "｠")
    # print("sss" , text)

    # 3. Anonymize the text, excluding protected numbers
    for pattern in patterns:
        # Skip the exclusion pattern (if any)
        if pattern.startswith("!"):
            continue

        # Find all matches of the current pattern
        matches = list(re.finditer(pattern, text))

        # Iterate through matches in reverse order to avoid index issues
        for match in reversed(matches):
            # Replace the match with a random string
            text = text[:match.start()] + generate_secure_random_string() + text[match.end():]

    # 4. Restore the protected numbers
    for i, num in enumerate(bracketed_numbers):
        text = text.replace(f"｟_PROTECTED_NUMBER_{i}｠", num, 1)

    # 5. Revert special symbols back to brackets
    text = text.replace("｟", "[")
    text = text.replace("｠", "]")
    return text

In [ ]:
def anonymize_PROPN(doc):
  """Anonymizes a spaCy Doc object by replacing sensitive tokens with anonymized versions."""
  new_tokens = []
  org_tokens = []
  # Store the whitespaces of original tokens
  whitespaces = [token.whitespace_ for token in doc]
  # Anonymize tokens based on their POS tags
  i = 0

  for token in doc:
    i +=1
    if i < 8:
      continue
    print(token.text)
    if token.pos_ == "PROPN":
      if token.tag_ == "名詞-数詞":
        new_tokens.append(token.text)
      elif token.tag_ == "接尾辞-名詞的-一般":
        new_tokens.append(token.text)
      elif token.tag_ == "感動詞-フィラー":
        new_tokens.append(token.text)
      elif token.tag_ == "名詞-サ変接続":
        new_tokens.append(token.text)
      elif token.tag_ == "名詞-普通名詞-一般":
        new_tokens.append(token.text)
      else:
        # print(token.text)
        # print(generate_secure_random_string())
        new_tokens.append(generate_secure_random_string())
        org_tokens.append(token.text) # Keep original token
    else:
      new_tokens.append(token.text)
  # print(len(org_tokens))
  # Check if org_tokens is not empty before accessing elements
  # if new_tokens:
    # print(len(new_tokens))
  # Create a new Doc with anonymized tokens
  new_doc = spacy.tokens.Doc(doc.vocab, words=new_tokens, spaces=whitespaces) # using spaces parameter
  return new_doc

In [ ]:
def botu_anonymize_PROPN(doc):
  """Anonymizes a spaCy Doc object by replacing sensitive tokens with anonymized versions."""
  new_tokens = []
  org_tokens = []
  # Store the whitespaces of original tokens
  whitespaces = [token.whitespace_ for token in doc]
  # Anonymize tokens based on their POS tags
  for token in doc:
    # print(token.text)
    if token.pos_ == "PROPN":
      if token.tag_ == "名詞-数詞":
        new_tokens.append(token.text)
      elif token.tag_ == "接尾辞-名詞的-一般":
        new_tokens.append(token.text)
      elif token.tag_ == "感動詞-フィラー":
        new_tokens.append(token.text)
      elif token.tag_ == "名詞-サ変接続":
        new_tokens.append(token.text)
      elif token.tag_ == "名詞-普通名詞-一般":
        new_tokens.append(token.text)
      else:
        # print(token.text)
        # print(generate_secure_random_string())
        new_tokens.append(generate_secure_random_string())
        org_tokens.append(token.text) # Keep original token
    else:
      new_tokens.append(token.text)
  print(len(org_tokens))
  # Create a new Doc with anonymized tokens
  new_doc = spacy.tokens.Doc(doc.vocab, words=new_tokens, spaces=whitespaces) # using spaces parameter
  return new_doc

In [ ]:
# https://qiita.com/CaughC/items/a67a2c81833c
# 基本 - 複数文章の同時処理
import spacy
# !pip install ja-ginza-electra
print(spacy.prefer_gpu()) # Trueと表示されれば, GPUが使用されます．
if(spacy.prefer_gpu()):
  nlp = spacy.load('ja_ginza_electra')
else:
  nlp = spacy.load('ja_ginza')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


def delete_begin(text):
  pattern = r"\d.*[-]+>"
  text = re.sub(pattern, "", text)  # マッチした部分を空文字に置換
  return text

def create_list_from_txt(filetxt):
  with open(filetxt, "r", encoding="utf-8") as f:
    # 空行を削除
    lines = [line.strip() for line in f if line.strip()]  # 空行は無視
    lines2 = [delete_begin(line) for line in lines]
  return lines2

list_txt = []
list_txt = create_list_from_txt(file_txt)
# text を Doc クラスに変換する
list_of_docs =nlp.pipe(list_txt)  # custom_nlp_pipe関数を使用
# Convert the iterable to a list
# # # Convert the iterable to a list
anonymized_docs = [anonymize_PROPN(doc) for doc in list_of_docs]  # Convert the iterable to a list

False
と
か
から
何
か
診断
さ
れ
た
結果
が
、
そこ
で
刺さ
れ
た
の
だろう
と
いう
結果
だ
と
いう
こと
です
か
ね
。


ValueError: [E027] Arguments `words` and `spaces` should be sequences of the same length, or `spaces` should be left default at None. `spaces` should be a sequence of booleans, with True meaning that the word owns a ' ' character following it.

In [ ]:
print(anonymized_docs[7:10])

[[77.24s] 予約した時にも楽天月末セールと書いてありますし、mknpVoN50hA9さんがお付けになったお値段かなと、お付けになって企画なさったのかなと、, [80.58s] こちらも誤解するような。, [92.58s] とも書いてありますし、予約確認、それから前日の確認、そこでも手配旅行という文字はどこにも見当たらなかったんですね。]


In [ ]:
def select_tag_PROPN(doc):
  for token in doc:
    if token.pos_ == "PROPN":
      return token.tag_

# Example Code for Rule-Based Reclassification
def reclassify_numbers(doc):
  for token in doc:
    if token.pos_ == "PROPN" and token.text.replace(".", "", 1).isdigit():
      # print(token.tag_, token.text)
      token.pos_ = "NUM"
  return doc

"""
Text: もとの語
Lemma: 語の基本形
POS: シンプルな品詞タグ
Tag: 詳細な品詞タグ
Dep: 統語的依存関係；トークンどうしの関係
Shape: 語形；大文字小文字，カンマ／ピリオドの使い方，数字，記号などの分類
is alpha: 英字か否か
is stop: stop words リストに含まれるか否か；高頻出語か否か
"""
def delete_FAKE_PROPN(doc):
  for token in doc:
    if('月末' in token.text):
      print(token.text)
    if token.pos_ == "PROPN":
      if token.tag_ == "名詞-数詞":
        continue
      elif token.tag_ == "接尾辞-名詞的-一般":
        continue
      elif token.tag_ == "感動詞-フィラー":
        continue
      elif token.tag_ == "名詞-サ変接続":
        continue
      elif token.tag_ == "名詞-普通名詞-一般":
        continue
      else:
        # if token.text=='None':
        # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
          # continue
        return token.text

list_txt = []
list_txt = create_list_from_txt(file_txt)
# text を Doc クラスに変換する
list_of_docs =nlp.pipe(list_txt)  # custom_nlp_pipe関数を使用
list_token_pos = []
list_PROPN =[]
list_token_tag =[]
list_PROPN_NUM =[]
list_tag_PROPN =[]
for doc in list_of_docs:
  list_tag_PROPN.extend([select_tag_PROPN(doc) for token in doc])
  # Doc クラスは Token クラスのイテレーターになっている
  list_token_pos.extend([token.pos_ for token in doc] )
  list_PROPN.extend([delete_FAKE_PROPN(doc) for token in doc])
  list_token_tag.extend([token.tag_ for token in doc])
  list_PROPN_NUM.extend([reclassify_numbers(doc) for token in doc])

# Universal POS tags list
print(set(list_tag_PROPN))
print(set(list_token_pos))
print(set(list_PROPN))
print(set(list_token_tag))

list_tag_PROPN = [item for item in list_tag_PROPN if item is not None]  # Filter out None values from list_PROPN
list_PROPN = [item for item in list_PROPN if item is not None]  # Filter out None values from list_PROPN
print(list_tag_PROPN)
print(list_PROPN)
list_set_tag_PROPN = list(set(list_tag_PROPN))
[i for i in list_set_tag_PROPN if '名詞' not in i]
for prpon in list_set_tag_PROPN:
  if '名詞' not in prpon:
    print(prpon)

anonymized_docs = [anonymize_PROPN(doc) for doc in list_of_docs]  # Convert the iterable to a list
print(anonymized_docs[:4])

'\nText: もとの語\nLemma: 語の基本形\nPOS: シンプルな品詞タグ\nTag: 詳細な品詞タグ\nDep: 統語的依存関係；トークンどうしの関係\nShape: 語形；大文字小文字，カンマ／ピリオドの使い方，数字，記号などの分類\nis alpha: 英字か否か\nis stop: stop words リストに含まれるか否か；高頻出語か否か\n'

月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
月末
{'名詞-固有名詞-人名-一般', '名詞-固有名詞-人名-名', '名詞-固有名詞-人名-姓', '名詞-固有名詞-一般', '名詞-固有名詞-地名-一般', '名詞-固有名詞-地名-国', '名詞-普通名詞-一般', '名詞-数詞', '感動詞-フィラー', None}
{'PROPN', 'VERB', 'CCONJ', 'SCONJ', 'PRON', 'ADP', 'AUX', 'DET', 'INTJ', 'PUNCT', 'X', 'ADJ', 'ADV', 'NOUN', 'SYM', 'NUM', 'PART'}
{'高橋', '時藤', '横山', '楽天トラベル', 'ラスール', '寛', 'ジャパン', '広瀬', '楽天', '栗原', '東京都', '日本', '学研', None}
{'感動詞-一般', '助動詞', '補助記号-句点', '助詞-係助詞', '形容詞-非自立可能', '接尾辞-名詞的-副詞可能', '助詞-準体助詞', '名詞-普通名詞-助数詞可能', '補助記号-括弧閉', '名詞-普通名詞-副詞可能', '接尾辞-名詞的-一般', '形容詞-一般', '接尾辞-名詞的-助数詞', '名詞-数詞', '補助記号-読点', '副詞', '名詞-固有名詞-人名-名', '名詞-固有名詞-人名-姓', '名詞-固有名詞-地名-国', '接尾辞-形容詞的', '連体詞', '名詞-普通名詞-一般', '形状詞-一般', '補助記号-括弧開', '名詞-固有名詞-人名-一般', '補助記号-一般', '助詞-終助詞', '助詞-接続助詞', '名詞-普通名詞-サ変可能', '名詞-普通名詞-サ変形状詞可能', '接頭辞', '感動詞-フィラー', '名詞-固有名詞-一般', '名詞-普通名詞-形状詞可能', '名詞-固有名詞-地名-一般', '動詞-一般', '接続詞', '助詞-格助詞', '助詞-副助詞', '形状詞-助動詞語幹', '代名詞'

['感動詞-フィラー']

感動詞-フィラー
[]


In [ ]:
# 市区町村データ
def create_list_municipalities(file_path):
    """
    Reads a CSV file containing municipality data and returns a list of municipality names.
    Args:
        file_path (str): The path to the CSV file.
    Returns:
        list: A list of municipality names as strings.
    """
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File not found at path: {file_path}")
        return []  # Return an empty list if file not found

    # Find the column containing municipality names
    municipality_column = next((col for col in df.columns if '市区町村名' in col), None)

    if municipality_column is None:
        print("Error: Column containing '市区町村名' not found in the CSV file.")
        return []  # Return an empty list if column not found

    # Extract and clean municipality names
    municipalities = df[municipality_column].dropna().astype(str).tolist()

    return municipalities

In [ ]:
import logging

# ロガーの設定 (force=True を使用)
logging.basicConfig(
    level=logging.INFO,
    format='%(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler(out_file_txt, 'w', encoding='utf-8'),  # ファイルへのハンドラ (エンコーディングを指定)
    ],
    force=True  # 既存のハンドラーを強制的に削除
)

# ログ出力
for doc in anonymized_docs:
    logging.info(doc)
# for doc1 in anonymized_docs1:
#     logging.info(doc1)
# for doc2 in anonymized_docs2:
#     logging.info(doc2)

[7.88s] 病院の先生とかから何か診断された結果が、そこで刺されたのだろうという結果だということですかね。
[20.82s] それで年末年始で病院も空いていないし、ホテルさんも病院も紹介してくれなかったので、すぐに迅速な対応もしていただけなかったんです。
[38.22s] それから予約前から、お掃除だけは毎日お願いしますねと言ったら、滞りなくするので、安心してくださいというメールまでいただいているんですね、行く前に。
[46.02s] それでもお掃除もできていないし、手術交換もなさっていないということもありまして。
[50.52s] これ楽天さんのホテルの手配旅行とか募集型というお話されていたんですけど、
[56.40s] もともとお送りいただいた内容は手配旅行と募集型とどちらだったんですかね、結果的に。
[63.02s] それがどちらともわからなかったんですけど、向こうは手配旅行だと言い張るんですが、
[77.24s] 予約した時にも楽天月末セールと書いてありますし、MbyqmVfeAOf9さんがお付けになったお値段かなと、お付けになって企画なさったのかなと、
[80.58s] こちらも誤解するような。
[92.58s] とも書いてありますし、予約確認、それから前日の確認、そこでも手配旅行という文字はどこにも見当たらなかったんですね。
[109.14s] 普通、手配旅行が宿泊客側にデメリットが多いということがあるようなんですけど、そういうことも、そういう念押しもなく、
[114.40s] 予約をさせるようなシステムだったので。
[120.80s] 内容の方は大変な思いをされたということは、今お伺いされました。
[131.72s] 今、消費者様はずっとこの継続されている中で、楽天さんの方に、もしくはホテル側さんの方にお話をどのようにされて、
[136.10s] 何を今お話しとして、お年どころとしてお考えになられているということでございますか。
[139.36s] そうですね、ホテルさんだったら、
[140.80s] こちらは全然、
[144.82s] 別に当たり前に、普通に綺麗に掃除がされていて、
[148.82s] 普通にお出迎えしてもらって、
[152.82s] 楽しかった、ありがとうって言って帰りたかったんですね。
[153.82s] はい。
[163.8

In [ ]:
# Patterns for sensitive information
patterns_ascii = [
  r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}",  # Matches email addresses
  r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}",  # Matches IP addresses
  r"\d{3}-?\d{4}",  # Matches Japanese postcodes
  r"https?://[^\s]+" # Matches URLs
]
pattern_alphabet_name = [r"[a-zA-Z]{2,}" ]# Matches names (at least two words)

より安全でスマートな方法として、secrets モジュールを使用することをお勧めします。

理由：

secrets モジュールは、暗号学的に安全な乱数を生成するために設計されています。
random モジュールよりも予測不可能な乱数を生成するため、セキュリティの観点からより安全です。
secrets.choice() 関数を使用することで、指定された文字セットからランダムな文字を選択できます。

In [ ]:
import re
import secrets
import string

def generate_secure_random_string(length=12):
    """Generate a secure random string of specified length."""
    alphabet = string.ascii_letters + string.digits
    return ''.join(secrets.choice(alphabet) for i in range(length))

def anonymize_with_pattern(patterns, text):
    """
    Replaces matches of patterns with random strings,
    excluding those in brackets with specific time formats.
    """

    # 1. Protect numbers inside brackets with a temporary placeholder
    # (Before replacing brackets with special symbols)
    bracketed_numbers = re.findall(r"(?:\d+[:.]\d+|\d+[hms])", text) # Changed pattern to match original brackets
    # print("aaa", bracketed_numbers)
    for i, num in enumerate(bracketed_numbers):
        text = text.replace(num, f"[_PROTECTED_NUMBER_{i}]", 1)  # Changed placeholder format

    # 2. Replace brackets with special symbols to avoid regex conflicts
    text = text.replace("[", "｟")
    text = text.replace("]", "｠")
    # print("sss" , text)

    # 3. Anonymize the text, excluding protected numbers
    for pattern in patterns:
        # Skip the exclusion pattern (if any)
        if pattern.startswith("!"):
            continue

        # Find all matches of the current pattern
        matches = list(re.finditer(pattern, text))

        # Iterate through matches in reverse order to avoid index issues
        for match in reversed(matches):
            # Replace the match with a random string
            text = text[:match.start()] + generate_secure_random_string() + text[match.end():]

    # 4. Restore the protected numbers
    for i, num in enumerate(bracketed_numbers):
        text = text.replace(f"｟_PROTECTED_NUMBER_{i}｠", num, 1)

    # 5. Revert special symbols back to brackets
    text = text.replace("｟", "[")
    text = text.replace("｠", "]")
    return text

# Patterns for sensitive information
patterns_phone = [
    r"\b0\d{2,3}(-\d{1,4}-\d{4}|\d{8})\b",  # Full phone numbers with or without hyphens
    r"\b0\d{2,3}-\d{1,4}\b",  # Partial phone numbers (e.g., 080-1234)
    r"\b0\d{2,3}\b",  # Partial phone numbers (e.g., 080)
    r"(?<![\d｟])\d{4}(?![\d｠])",  # Matches 4-digit numbers not preceded or followed by other digits or brackets
    r"(?<![\d｟])\d{2}(?![\d｠])"   # Matches 2-digit numbers not preceded or followed by other digits or brackets
]

In [ ]:
text = "[1:23:45] お電話番号をどうぞ。[100.4s]\n070\nはい\n1234\nはい\n5678\nありがとうございます。070-3456-7890ですね\nちがいます。070-1234-5678です。はい34と 56ですね"
anonymized_text = anonymize_with_pattern(patterns_phone, text)
print(anonymized_text)

[1:23:45] お電話番号をどうぞ。[100.4s]
KaJGsB5dTqY3
はい
BD5dyFSFrx7k9zyjttJfl9
はい
a5ety5LQ9hcm
ありがとうございます。FElIcbO4entc2bCHFu0GSx-ehkeyq3XrHIlですね
ちがいます。md4W5MyCkRlQK6g7MjqNp4-o5Pv1sDl1QT8です。はい4lZGsS4QEeA5と KvKHAMKmyLjCですね


テスト用

generate_random_stringは、最初の関数がおすすめです。

理由：

よりランダム性が高い: 最初の関数は、大文字と数字の両方を使用するため、生成される文字列のバリエーションが豊富になり、よりランダム性が高まります。2番目の関数は大文字のみを使用するため、ランダム性が低くなります。
より安全: 最初の関数は、より多くの文字を使用するため、生成される文字列が推測されにくくなり、より安全です。
結論：

よりランダム性と安全性を考慮すると、最初の関数の使用が推奨されます。

In [ ]:
# def anonymize_doc(doc):
#   """Anonymizes a spaCy Doc object by replacing sensitive tokens with anonymized versions."""
#   new_tokens = []
#   print(doc)
#   # Store the whitespaces of original tokens
#   whitespaces = [token.whitespace_ for token in doc]
#   for token in doc:
#     if 'PRO' in token.pos_:  # Check if the token is a pronoun
#       print(token.text,token.pos_)
#       # new_tokens.append(generate_secure_random_string())
#     # elif 'NUM' in token.pos_:  # Check if the token is a number
#       pattern_num = r"(?:\d+[:.]\d+|\d+[hms])"
#       compiled_pattern = re.compile(pattern_num, re.UNICODE)
#       print(compiled_pattern)
#       if compiled_pattern:
#         print(token.text,token.pos_)
#       pattern = r"^[^ -~｡-ﾟ]+$"
#       compiled_pattern2 = re.compile(pattern_num, re.UNICODE)
#       if compiled_pattern2:
#         print(token.text,token.pos_)
#         # new_tokens.append(generate_secure_random_string())
#     else:
#       new_tokens.append(token.text)  # Keep original token

#   # Create a new Doc with anonymized tokens
#   new_doc = spacy.tokens.Doc(doc.vocab, words=new_tokens, spaces=whitespaces) # using spaces parameter
#   return new_doc

In [ ]:
# # # Convert the iterable to a list
# anonymized_docs = [anonymize_with_pattern(patterns_phone, doc.text) for doc in list_of_docs]  # Convert the iterable to a list

# # Change here: Access the .text attribute of the Doc object before passing it to the function
#The original problem is that you're applying the anonymization twice, first in the creation of
#anonymized_docs and then again in anonymized_docs1. Since anonymized_docs is already a list of strings,
#you should apply anonymize_interview_data_ascii directly to those strings.

patterns_ascii = [
    r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}",  # Email addresses
    r"\b(?:\d{1,3}\.){3}\d{1,3}\b",  # IP addresses
    # Add other patterns as needed
]

anonymized_docs2 = [anonymize_interview_data_ascii(doc, patterns_ascii) for doc in anonymized_docs1]


In [ ]:
# Anonymizes Japanese names
def anonymize_interview_data(text):
    """
    Anonymizes interview data by replacing sensitive information with random strings.
    Args:
        text (str): The interview text to anonymize.
    Returns:
        str: The anonymized interview text.
    """

    # # Patterns for sensitive information (add or modify as needed)
    # patterns = {
    #     "address": r"[\w\s,-]+",  # Matches general address patterns
    #     "institution": r"[^\s]+(?:\s[^\s]+)+",  # Matches institution names (at least two words)
    #

    # Replace municipality names using the create_list_municipalities function
    municipalities = create_list_municipalities('./20240101現在の団体.csv')
    municipalities.sort(key=len, reverse=True)
    for municipality in municipalities:
        text = re.sub(re.escape(municipality), generate_secure_random_string(), text)

    # # Anonymize other sensitive information using the defined patterns
    # for pattern_type, pattern in patterns.items():
    #     compiled_pattern = re.compile(pattern)  # Compile pattern for efficiency
    #     # text = anonymize_pattern(compiled_pattern, text)
    #     text = anonymize_with_pattern(compiled_pattern, text)
    return text

In [ ]:
def anonymize_doc(doc):
  """Anonymizes a spaCy Doc object by replacing sensitive tokens with anonymized versions."""
  new_tokens = []
  # Store the whitespaces of original tokens
  whitespaces = [token.whitespace_ for token in doc]
  # Anonymize tokens based on their POS tags
  for token in doc:
    if token.pos_ == 'PRON': # Replace 'PRON' with the actual POS tag for pronouns in your data
      print(token.pos_, token.text)
      # Anonymize pronouns, ensuring a non-empty string is returned
      anonymized_token = generate_secure_random_string()
      # If anonymized_token is empty, keep the original token
      new_tokens.append(anonymized_token if anonymized_token else token.text)
    else:
      new_tokens.append(token.text)  # Keep original token

  # Create a new Doc with anonymized tokens
  # Handle case where new_tokens is empty
  if new_tokens:
    new_doc = spacy.tokens.Doc(doc.vocab, words=new_tokens, spaces=whitespaces)
  else:
    # Return original doc or handle appropriately
    new_doc = doc  # Or raise an exception, log a message, etc.

  return new_doc

In [ ]:
import re

def anonymize_interview_data_ascii(text, patterns):
    """
    Anonymizes interview data by replacing sensitive information with random strings.
    Focuses on numeric, alphabets, and patterns like email addresses and URLs.
    Args:
        text (str): The interview text to anonymize.
    Returns:
        str: The anonymized interview text.
    """
    # Anonymize using the defined patterns
    for pattern in patterns:
        # This ensures the pattern treats the string as unicode
        # This may help in correctly interpreting double-byte characters.
        # Compile the pattern with the re.UNICODE flag
        compiled_pattern = re.compile(pattern, re.UNICODE)
        # print(compiled_pattern)

        # Pass the original pattern and the compiled pattern to anonymize_with_pattern
        text = anonymize_with_pattern_ascii(pattern, compiled_pattern, text)

    return text

def anonymize_with_pattern_ascii(pattern, compiled_pattern, text):
    """
    Anonymizes text using the provided patterns.

    Args:
        pattern (str): The original pattern string.
        compiled_pattern (_sre.SRE_Pattern): The compiled regex pattern.
        text (str): The text to anonymize.

    Returns:
        str: The anonymized text.
    """
    # Skip the exclusion pattern (if any)
    if pattern.startswith("!"):
        return text  # Return the original text if it's an exclusion pattern

    # Perform anonymization using the compiled pattern
    # ... (your existing logic to anonymize text using compiled_pattern) ...

    return text  # Return the anonymized text

Here's the breakdown of the issue:

anonymized_docs is a list of strings, as revealed by the global variables you provided: anonymized_docs value: 4 items ['[435.44s] 今日の申し出の内容だ…', ... ,type: list.
The list comprehension iterates through each item (doc) in anonymized_docs.
Inside the comprehension, you attempt to access doc.text, but doc is a string, not a spaCy Doc object (which would have a text attribute). This is why the AttributeError is raised.
Essentially, you are expecting anonymized_docs to be a list of spaCy Doc objects, but it's actually a list of strings, likely already processed text.



最終的な推奨コード

「解決策1：force=True を使用する」方がおすすめです。

理由

force=True を使用することで、既存のハンドラーを強制的に削除し、新しいハンドラーを設定することができます。これにより、ハンドラーの競合を防ぎ、ログが正しくファイルに書き込まれることを保証できます。
logging.shutdown() を使用してバッファをフラッシュする方法は、ログ出力の最後にのみ実行する必要があります。force=True を使用すると、この手順を省略することができます。
force=True は Python 3.8 以降で利用可能であり、より新しいバージョンで推奨される方法です。

In [ ]:
# import logging

# logging.basicConfig(
#     level=logging.INFO,
#     format='%(message)s',
#     handlers=[
#         logging.StreamHandler(),
#         logging.FileHandler('output.txt', 'w', encoding='utf-8'),  # ファイルへのハンドラ (エンコーディングを指定)
#     ],
#     force=True  # 既存のハンドラーを強制的に削除
# )

# # ログ出力
# for doc in anonymized_docs2:
#     logging.info(doc)

解決策2：ロガーを明示的に取得する で同じ行が二度出力されたとのこと、申し訳ございません。これは、ルートロガーと明示的に取得したロガーの両方にハンドラーが設定されているため、ログが重複して出力されたことが原因と考えられます。

In [ ]:
# import logging

# # ロガーの取得
# logger = logging.getLogger(__name__)  # 現在のモジュール名を使用
# logger.setLevel(logging.INFO)

# # ハンドラーの設定
# file_handler = logging.FileHandler('output.txt', 'w', encoding='utf-8')
# file_handler.setLevel(logging.INFO)
# file_handler.setFormatter(logging.Formatter('%(message)s'))

# stream_handler = logging.StreamHandler()
# stream_handler.setLevel(logging.INFO)
# stream_handler.setFormatter(logging.Formatter('%(message)s'))

# # ロガーにハンドラーを追加
# logger.addHandler(file_handler)
# logger.addHandler(stream_handler)

# # ログ出力
# for doc in anonymized_docs2:
#     logger.info(doc)

In [ ]:
# import logging

# # ロガーの設定 (force=True を使用)
# logging.basicConfig(
#     level=logging.INFO,
#     format='%(message)s',
#     handlers=[
#         logging.StreamHandler(),
#         logging.FileHandler('output.txt', 'w', encoding='utf-8'),  # ファイルへのハンドラ (エンコーディングを指定)
#     ],
#     force=True
# )

# # ログ出力
# for doc in anonymized_docs2:
#     logging.info(doc)

# # バッファをフラッシュ
# logging.shutdown()  # すべてのハンドラーをクローズし、バッファをフラッシュ

In [ ]:
# import spacy

# # print(spacy.prefer_gpu()) # Trueと表示されれば, GPUが使用されます．
# if(spacy.prefer_gpu()):
#   nlp = spacy.load('ja_ginza_electra')
# else:
#   nlp = spacy.load('ja_ginza')
# doc = nlp('山田太郎は東京都渋谷区に住んでいます。木村 花子は山口に住んでいましたが、現在は引っ越して桐生に住んでいます。山口は近々、仕事の都合で津に転居するという話です。あ、三重県の津市のことです。')

# print("go!")

# for ent in doc.ents:
# 	if ent.label_ == 'Person':
# 		print('人名:', ent.text)
# 	elif ent.label_ == 'City':
# 		print('住所:', ent.text)


In [ ]:
import difflib

In [ ]:
pwd = os.getcwd()
pwd

In [ ]:
def compare_files(file1_path, file2_path):
    with open(file1_path, 'r') as file1, open(file2_path, 'r') as file2:
        diff = difflib.ndiff(file1.readlines(), file2.readlines())
        print(''.join(diff))

In [ ]:
# file1_path = './入力フォルダのパス/20240420_1546中央郵便.txt'
# file2_path = './出力フォルダのパス/20240420_1546中央郵便.txt'
# compare_files(file1_path, file2_path)

In [ ]:
# import spacy

# nlp = spacy.load('ja_ginza')
# doc = nlp('山田太郎は東京都渋谷区に住んでいます。木村 花子は山口に住んでいましたが、現在は引っ越して桐生に住んでいます。山口は近々、仕事の都合で津に転居するという話です。あ、三重県の津市のことです。')

# print("go!")

# for ent in doc.ents:
# 	if ent.label_ == 'Person':
# 		print('人名:', ent.text)
# 	elif ent.label_ == 'City':
# 		print('住所:', ent.text)
